In [1]:
from surface_dynamics.all import Origami
import numpy as np

In [2]:
def cycle_notation(array): 
    '''
        Argument: array of integers representing gluing instruction of origami, every element in array is glued to the next
        Return: string representing the cycle of the gluing
        Comment: assumes array is a permutation we are using which is a cycle as we are gluing the boxes
            until the edges meet, and is therefore NOT a general method for converting permutation to cycle notation, 
            it basically makes the array a string
    '''
    cycle = '('
    for i in array:
        cycle = cycle + str(int(i)) + ','
    cycle = cycle[:-1] + ')'
    return cycle

In [3]:
'''
    Type 1 diagram:
    □ <- rect1
    □□ <- rect2
     □□ <- rect3
'''

class H4Type1:
    def __init__(self, h1, u1, h2, u2, h3, u3):
        self.h1 = h1
        self.u1 = u1
        self.h2 = h2
        self.u2 = u2
        self.h3 = h3
        self.u3 = u3
        rectangle1 = np.arange(1, u1 * h1 + 1)
        rectangle1 = np.reshape(rectangle1, (h1, u1))
        self.rect1 = np.flip(rectangle1, 0)
        rectangle2 = np.arange(u1 * h1 + 1, u1 * h1 + u2 * h2 + 1)
        rectangle2 = np.reshape(rectangle2, (h2, u2))
        self.rect2 = np.flip(rectangle2, 0)
        rectangle3 = np.arange(u2 * h2 + u1 * h1 + 1, u2 * h2 + u1 * h1 + u3 * h3 + 1)
        rectangle3 = np.reshape(rectangle3, (h3, u3))
        self.rect3 = np.flip(rectangle3, 0)
        self.hperm = '' 
        # horizontal (right) permutation is independent of twist
        for i in range(h1):
            self.hperm += cycle_notation(self.rect1[i])
        for i in range(h2):
            self.hperm += cycle_notation(self.rect2[i])
        for i in range(h3):
            self.hperm += cycle_notation(self.rect3[i])
        
    def Type1Cylinder1(self, offset1, currentperm):
        '''
            Argument: the offset to start with, 
                      the current accumulating permutation
            Return: the permutation in list(array) form BEFORE flip
            Comment: the parameters are supposed to be passed by REFERENCE
        '''
        if self.marker1[offset1] == 0:
            return currentperm
        permutation = np.concatenate(([row[offset1] for row in self.rect1], currentperm))
        self.marker1[offset1] = 0
        offset2 = (offset1 - self.t1 + self.u1) % self.u1
        return self.Type1Cylinder2(offset2, permutation)

    def Type1Cylinder2(self, offset2, currentperm):
        if self.marker2[offset2] == 0:
            return currentperm
        permutation = np.concatenate(([row[offset2] for row in self.rect2], currentperm))
        self.marker2[offset2] = 0
        if (offset2 - self.t2 + self.u2) % self.u2 < self.u1:
            offset1 = (offset2 - self.t2 + self.u2) % self.u2
            return self.Type1Cylinder1(offset1, permutation)
        else:
            offset3 = ((offset2 - self.t2 + self.u2) % self.u2 - self.u1 + self.t3 + self.u3 - (self.u2 - self.u1)) % self.u3
            return self.Type1Cylinder3(offset3, permutation)

    def Type1Cylinder3(self, offset3, currentperm):
        if self.marker3[offset3] == 0:
            return currentperm
        permutation = np.concatenate(([row[offset3] for row in self.rect3], currentperm))
        self.marker3[offset3] = 0
        if offset3 < self.u2 - self.u1:
            offset2 = (self.u1 + offset3) % self.u2
            return self.Type1Cylinder2(offset2, permutation)
        else:
            offset3 = (self.t3 + offset3 - (self.u2 - self.u1)) % self.u3
            return self.Type1Cylinder3(offset3, permutation)

    def Constr_Ori(self, t1, t2, t3):
        self.marker1 = [1] * self.u1 # marks columns in cylinder that have been visited by 0
        self.marker2 = [1] * self.u2
        self.marker3 = [1] * self.u3
        self.t1 = t1
        self.t2 = t2
        self.t3 = t3
        self.vperm = ''
        while sum(self.marker2) != 0:
            offset2 = np.nonzero(self.marker2)[0][0]
            column = []
            permutation = self.Type1Cylinder2(offset2, column)
            self.vperm += cycle_notation(np.flip(permutation,0))
        while sum(self.marker1) != 0:
            offset1 = np.nonzero(self.marker1)[0][0]
            column = []
            permutation = self.Type1Cylinder1(offset1, column)
            self.vperm += cycle_notation(np.flip(permutation,0))
        while sum(self.marker3) != 0:
            offset3 = np.nonzero(self.marker3)[0][0]
            column = []
            permutation = self.Type1Cylinder3(offset3, column)
            self.vperm += cycle_notation(np.flip(permutation,0))
        origami = Origami(self.hperm, self.vperm)
        return origami

In [4]:
def enum_H4Type1(n):
    '''
        Argument: integer n, size of origamis
        Return: array of origamis of the form type 1 in H(4) that have size n
        Comment: https://jamboard.google.com/d/1OSRaahlM2KiY1_onnQ5y6TDBOrseFGrHhtssKQc2ee0/edit?usp=sharing
            on slide 6 explains external gluing formula
    '''
    surfaces = []
    for u2 in range(2, n): # middle rectangle has width [2, n-1)
        for h2 in range(1, n // u2 + 1): # middle rectangle has height [1, n/u2)
            # top rectangle has width at most u2 - 1 or at most area left - 2
            for u1 in range(1, min(u2, n - u2 * h2 - 1)):
                for h1 in range(1, (n - u2 * h2) // u1 + 1): # top rectangle has width [1, area)
                    for u3 in range(u2 - u1 + 1, n - u2 * h2 - u1 * h1 + 1):
                        if (n - u2 * h2 - u1 * h1) % u3 == 0: # as long as u1 divides the area, it's valid
                            h3 = (n - u2 * h2 - u1 * h1) // u3
                            
                            Cylinders = H4Type1(h1, u1, h2, u2, h3, u3)
                            for t1 in range(u1): # singularity on cylinder 1
                                for t2 in range(u2): # singularity on cylinder 2
                                    for t3 in range(u3):
                                        surfaces.append(Cylinders.Constr_Ori(t1, t2, t3))
    return surfaces

In [5]:
for i in range(1):
    for ori in enum_H4Type1(i):
        print(ori.stratum())

In [6]:
'''
    Type 2 diagram:
    rect1 -> □ □ <- rect3
    rect2 -> □□□
'''

class H4Type2:
    # Note for future: some sort of inheritance can certainly be used to avoid code repetition
    def __init__(self, h1, u1, h2, u2, h3, u3):
        self.h1 = h1
        self.u1 = u1
        self.h2 = h2
        self.u2 = u2
        self.h3 = h3
        self.u3 = u3
        rectangle1 = np.arange(1, u1 * h1 + 1)
        rectangle1 = np.reshape(rectangle1, (h1, u1))
        self.rect1 = np.flip(rectangle1, 0)
        rectangle2 = np.arange(u1 * h1 + 1, u1 * h1 + u2 * h2 + 1)
        rectangle2 = np.reshape(rectangle2, (h2, u2))
        self.rect2 = np.flip(rectangle2, 0)
        rectangle3 = np.arange(u2 * h2 + u1 * h1 + 1, u2 * h2 + u1 * h1 + u3 * h3 + 1)
        rectangle3 = np.reshape(rectangle3, (h3, u3))
        self.rect3 = np.flip(rectangle3, 0)
        self.hperm = '' 
        # horizontal (right) permutation is independent of twist
        for i in range(h1):
            self.hperm += cycle_notation(self.rect1[i])
        for i in range(h2):
            self.hperm += cycle_notation(self.rect2[i])
        for i in range(h3):
            self.hperm += cycle_notation(self.rect3[i])
        
    def Type2Cylinder1(self, offset1, currentperm):
        '''
            Argument: the offset to start with, 
                      the current accumulating permutation
            Return: the permutation in list(array) form BEFORE flip
            Comment: the parameters are supposed to be passed by REFERENCE
        '''
        if self.marker1[offset1] == 0:
            return currentperm
        permutation = np.concatenate(([row[offset1] for row in self.rect1], currentperm))
        self.marker1[offset1] = 0
        offset2 = ((offset1 - self.t1 + self.u1) % self.u1 + self.t2 + self.u2 - self.u1) % self.u2
        return self.Type2Cylinder2(offset2, permutation)

    def Type2Cylinder2(self, offset2, currentperm):
        if self.marker2[offset2] == 0:
            return currentperm
        permutation = np.concatenate(([row[offset2] for row in self.rect2], currentperm))
        self.marker2[offset2] = 0
        if offset2 < self.u1:
            offset1 = offset2
            return self.Type2Cylinder1(offset1, permutation)
        elif offset2 < self.u2 - self.u3:
            offset2 = (offset2 - self.u1 + self.t2 + self.u3) % self.u2
            return self.Type2Cylinder2(offset2, permutation)
        else:
            offset3 = offset2 - (self.u2 - self.u3)
            return self.Type2Cylinder3(offset3, permutation)
    
    def Type2Cylinder3(self, offset3, currentperm):
        if self.marker3[offset3] == 0:
            return currentperm
        permutation = np.concatenate(([row[offset3] for row in self.rect3], currentperm))
        self.marker3[offset3] = 0
        offset2 = ((offset3 - self.t3 + self.u3) % self.u3 + self.t2) % self.u2
        return self.Type2Cylinder2(offset2, permutation)

    def Constr_Ori(self, t1, t2, t3):
        self.marker1 = [1] * self.u1 # marks columns in cylinder that have been visited by 0
        self.marker2 = [1] * self.u2
        self.marker3 = [1] * self.u3
        self.t1 = t1
        self.t2 = t2
        self.t3 = t3
        self.vperm = ''
        while sum(self.marker2) != 0:
            offset2 = np.nonzero(self.marker2)[0][0]
            column = []
            permutation = self.Type2Cylinder2(offset2, column)
            self.vperm += cycle_notation(np.flip(permutation,0))
        while sum(self.marker1) != 0:
            offset1 = np.nonzero(self.marker1)[0][0]
            column = []
            permutation = self.Type2Cylinder1(offset1, column)
            self.vperm += cycle_notation(np.flip(permutation,0))
        while sum(self.marker3) != 0:
            offset3 = np.nonzero(self.marker3)[0][0]
            column = []
            permutation = self.Type2Cylinder3(offset3, column)
            self.vperm += cycle_notation(np.flip(permutation,0))
        origami = Origami(self.hperm, self.vperm)
        return origami

In [7]:
def enum_H4Type2(n):
    '''
        Argument: integer n, size of origamis
        Return: array of origamis of the form type 2 in H(4) that have size n
        Comment: https://jamboard.google.com/d/1OSRaahlM2KiY1_onnQ5y6TDBOrseFGrHhtssKQc2ee0/edit?usp=sharing
            on slide 7 explains parameters
    '''
    surfaces = []
    for u2 in range(3, n - 1): # middle rectangle has width [2, n-1)
        for h2 in range(1, n // u2 + 1): # middle rectangle has height [1, n/u2)
            # top rectangle has width at most u2 - 2 or at most area left - 1
            for u1 in range(1, min(u2 - 1, n - u2 * h2)):
                for h1 in range(1, (n - u2 * h2) // u1 + 1): # top rectangle has width [1, area)
                    for u3 in range(1, min(n - u2 * h2 - u1 * h1 + 1, u2 - u1)):
                        if (n - u2 * h2 - u1 * h1) % u3 == 0: # as long as u1 divides the area, it's valid
                            h3 = (n - u2 * h2 - u1 * h1) // u3
                            Cylinders = H4Type2(h1, u1, h2, u2, h3, u3)
                            for t1 in range(u1): # singularity on cylinder 1
                                for t2 in range(u2): # singularity on cylinder 2
                                    for t3 in range(u3):
                                        surfaces.append(Cylinders.Constr_Ori(t1, t2, t3))
    return surfaces

In [8]:
for i in range(1):
    for ori in enum_H4Type2(i):
        print(ori.stratum())

In [9]:
'''
    Type 3 diagram:
    □ <- rect1
    □□□□ <- rect2
'''

class H4Type3:
    def __init__(self, h1, u1, h2, u2):
        self.h1 = h1
        self.u1 = u1
        self.h2 = h2
        self.u2 = u2
        rectangle1 = np.arange(1, u1 * h1 + 1)
        rectangle1 = np.reshape(rectangle1, (h1, u1))
        self.rect1 = np.flip(rectangle1, 0)
        rectangle2 = np.arange(u1 * h1 + 1, u1 * h1 + u2 * h2 + 1)
        rectangle2 = np.reshape(rectangle2, (h2, u2))
        self.rect2 = np.flip(rectangle2, 0)
        self.hperm = '' 
        # horizontal (right) permutation is independent of twist
        for i in range(h1):
            self.hperm += cycle_notation(self.rect1[i])
        for i in range(h2):
            self.hperm += cycle_notation(self.rect2[i])
        
    def Type3Cylinder1(self, offset1, currentperm):
        '''
            Argument: the offset to start with, 
                      the current accumulating permutation
            Return: the permutation in list(array) form BEFORE flip
            Comment: the parameters are supposed to be passed by REFERENCE
        '''
        if self.marker1[offset1] == 0:
            return currentperm
        permutation = np.concatenate(([row[offset1] for row in self.rect1], currentperm))
        self.marker1[offset1] = 0
        offset2 = (self.t2 + (self.u1 + offset1 - self.t1) % self.u1) % self.u2
        return self.Type3Cylinder2(offset2, permutation)

    def Type3Cylinder2(self, offset2, currentperm):
        if self.marker2[offset2] == 0:
            return currentperm
        permutation = np.concatenate(([row[offset2] for row in self.rect2], currentperm))
        self.marker2[offset2] = 0
        if offset2 < self.u1:
            offset1 = offset2
            return self.Type3Cylinder1(offset1, permutation)
        else:
            offset = 0
            if offset2 - self.u1 >= self.w2 + self.w1:
                offset += (offset2 - self.u1 - self.w2 - self.w1)
            else:
                offset += self.w3
                if offset2 - self.u1 >= self.w1:
                    offset += (offset2 - self.u1 - self.w1)
                else:
                    offset += (self.w2 + offset2 - self.u1)
            offset2 = (self.t2 + self.u1 + offset) % self.u2
            return self.Type3Cylinder2(offset2, permutation)

    def Constr_Ori(self, w1, w2, t1, t2):
        self.marker1 = [1] * self.u1 # marks columns in cylinder that have been visited by 0
        self.marker2 = [1] * self.u2
        self.w1 = w1
        self.w2 = w2
        self.w3 = self.u2 - self.u1 - w1 - w2
        self.t1 = t1
        self.t2 = t2
        self.vperm = ''
        while sum(self.marker2) != 0:
            offset2 = np.nonzero(self.marker2)[0][0]
            column = []
            permutation = self.Type3Cylinder2(offset2, column)
            self.vperm += cycle_notation(np.flip(permutation,0))
        while sum(self.marker1) != 0:
            offset1 = np.nonzero(self.marker1)[0][0]
            column = []
            permutation = self.Type3Cylinder1(offset1, column)
            self.vperm += cycle_notation(np.flip(permutation,0))
        origami = Origami(self.hperm, self.vperm)
        return origami

In [10]:
def enum_H4Type3(n):
    '''
        Argument: integer n, size of origamis
        Return: array of origamis of the form type 3 in H(4) that have size n
        Comment: https://jamboard.google.com/d/1OSRaahlM2KiY1_onnQ5y6TDBOrseFGrHhtssKQc2ee0/edit?usp=sharing
            on slide 4 explains external gluing formula
    '''
    surfaces = []
    for u2 in range(4, n): # base rectangle has width [4, n-1)
        for h2 in range(1, n // u2 + 1): # base rectangle has height [1, n/u2)
            # top rectangle has width at most u2 - 3 or at most area of the top rectangle
            upperbound = min(u2 - 2, n - u2 * h2 + 1) 
            for u1 in range(1, upperbound): # top rectangle has width [1, upperbound)
                if (n - u2 * h2) % u1 == 0: # as long as u1 divides the area, it's valid
                    h1 = (n - u2 * h2) // u1
                    Cylinders = H4Type3(h1, u1, h2, u2)
                    # width of smaller rectangles
                    for w1 in range(1, u2 - u1 - 1): # w1 is between 1 and u2-u1-2
                        for w2 in range(1, u2 - u1 - w1): # w2 is between 1 and u2-u1-w1-1
                            for t1 in range(u1): # singularity on cylinder 1
                                for t2 in range(u2): # singularity on cylinder 2
                                    surfaces.append(Cylinders.Constr_Ori(w1, w2, t1, t2))
    return surfaces

In [11]:
for i in range(1):
    for ori in enum_H4Type3(i):
        print(ori.stratum())

In [12]:
'''
    Type 4 diagram:
    □□
     □□□
'''

class H4Type4:
    def __init__(self, h1, u1, h2, u2):
        self.h1 = h1
        self.u1 = u1
        self.h2 = h2
        self.u2 = u2
        rectangle1 = np.arange(1, u1 * h1 + 1)
        rectangle1 = np.reshape(rectangle1, (h1, u1))
        self.rect1 = np.flip(rectangle1, 0)
        rectangle2 = np.arange(u1 * h1 + 1, u1 * h1 + u2 * h2 + 1)
        rectangle2 = np.reshape(rectangle2, (h2, u2))
        self.rect2 = np.flip(rectangle2, 0)
        self.hperm = '' 
        # horizontal (right) permutation is independent of twist
        for i in range(h1):
            self.hperm += cycle_notation(self.rect1[i])
        for i in range(h2):
            self.hperm += cycle_notation(self.rect2[i])
        
    def Type4Cylinder1(self, offset1, currentperm):
        '''
            Argument: the offset to start with, 
                      the current accumulating permutation
            Return: the permutation in list(array) form BEFORE flip
            Comment: the parameters are supposed to be passed by REFERENCE
        '''
        if self.marker1[offset1] == 0:
            return currentperm
        permutation = np.concatenate(([row[offset1] for row in self.rect1], currentperm))
        self.marker1[offset1] = 0
        if (offset1 + self.u1 - self.t1) % self.u1 >= self.v:
            offset1 = (offset1 + self.u1 - self.t1 - self.v) % self.u1
            return self.Type4Cylinder1(offset1, permutation)
        else:
            offset2 = (self.t2 + (offset1 + self.u1 - self.t1) % self.u1) % self.u2
            return self.Type4Cylinder2(offset2, permutation)

    def Type4Cylinder2(self, offset2, currentperm):
        if self.marker2[offset2] == 0:
            return currentperm
        permutation = np.concatenate(([row[offset2] for row in self.rect2], currentperm))
        self.marker2[offset2] = 0
        if offset2 >= self.v:
            if (offset2 - self.v) % self.u2 >= self.w1:
                offset2 = (self.t2 + offset2 - self.w1) % self.u2
            else:
                offset2 = (self.t2 + self.w2 + offset2) % self.u2
            return self.Type4Cylinder2(offset2, permutation)
        else:
            offset1 = (self.u1 - self.v + offset2) % self.u1
            return self.Type4Cylinder1(offset1, permutation)
        
    def Constr_Ori(self, v, w1, t1, t2):
        self.marker1 = [1] * self.u1 # marks columns in cylinder that have been visited by 0
        self.marker2 = [1] * self.u2
        self.v = v
        self.w1 = w1
        self.w2 = self.u2 - v - w1
        self.t1 = t1
        self.t2 = t2
        self.vperm = ''
        while sum(self.marker2) != 0:
            offset2 = np.nonzero(self.marker2)[0][0]
            column = []
            permutation = self.Type4Cylinder2(offset2, column)
            self.vperm += cycle_notation(np.flip(permutation,0))
        while sum(self.marker1) != 0:
            offset1 = np.nonzero(self.marker1)[0][0]
            column = []
            permutation = self.Type4Cylinder1(offset1, column)
            self.vperm += cycle_notation(np.flip(permutation,0))
        origami = Origami(self.hperm, self.vperm)
        return origami

In [13]:
def enum_H4Type4(n):
    '''
        Argument: integer n, size of origamis
        Return: array of origamis of the form type 4 in H(4) that have size n
        Comment: https://jamboard.google.com/d/1OSRaahlM2KiY1_onnQ5y6TDBOrseFGrHhtssKQc2ee0/edit?usp=sharing
            on slide 5 explains external gluing formula
    '''
    surfaces = []
    for u2 in range(3, n - 1): # base rectangle has width [3, n-2), because top rectangle is at least 2
        for h2 in range(1, n // u2 + 1): # base rectangle has height [1, n/u2)
            # top rectangle has width at most u2 - 3 or at most area of the top rectangle
            for v in range(1, u2 - 1): # v is in [1, u2-2]
                for u1 in range(v + 1, n - u2 * h2 + 1): # top rectangle has width [v+1, area)
                    if (n - u2 * h2) % u1 == 0: # as long as u1 divides the area, it's valid
                        h1 = (n - u2 * h2) // u1
                        Cylinders = H4Type4(h1, u1, h2, u2)
                        # width of smaller rectangles
                        for w1 in range(1, u2 - v): # w1 is between 1 and u2-v-1
                            w2 = u2 - v - w1
                            for t1 in range(u1): # singularity on cylinder 1
                                for t2 in range(u2): # singularity on cylinder 2
                                    surfaces.append(Cylinders.Constr_Ori(v, w1, t1, t2))
    return surfaces

In [14]:
for i in range(1):
    for ori in enum_H4Type4(i):
        print(ori.stratum())

In [15]:
def enum_H4(n):
    oris = enum_H4Type1(n)
    oris.extend(enum_H4Type2(n))
    oris.extend(enum_H4Type3(n))
    oris.extend(enum_H4Type4(n))
    return oris

In [16]:
def count_isom_classes(oris):
    counted = []
    for ori in oris:
        is_counted = False
        for compare in counted:
            if ori.is_isomorphic(compare):
                is_counted = True
                break
        if not is_counted:
            counted.append(ori)
    return len(counted)

In [17]:
%%time
print(len(enum_H4(17)))

52402
CPU times: user 47.2 s, sys: 375 ms, total: 47.6 s
Wall time: 48.3 s


In [18]:
%%time
print(count_isom_classes(enum_H4(11)))

4263
CPU times: user 27.6 s, sys: 62 ms, total: 27.7 s
Wall time: 28 s


In [19]:
%%time
print(len(enum_H4Type1(26)))

81143
CPU times: user 1min 44s, sys: 1.27 s, total: 1min 46s
Wall time: 1min 46s


In [20]:
%%time
print(len(enum_H4Type2(17)))

7957
CPU times: user 8.72 s, sys: 15 ms, total: 8.73 s
Wall time: 8.88 s


In [21]:
%%time
print(len(enum_H4Type3(17)))

16151
CPU times: user 19.2 s, sys: 47 ms, total: 19.2 s
Wall time: 19.4 s


In [22]:
%%time
print(len(enum_H4Type4(17)))

19476
CPU times: user 27.1 s, sys: 156 ms, total: 27.2 s
Wall time: 27.4 s


In [23]:
%%time
for i in range(13):
    print("i = ", i, ", ", len(enum_H4(i)), ":", count_isom_classes(enum_H4(i)))

i =  0 ,  0 : 0
i =  1 ,  0 : 0
i =  2 ,  0 : 0
i =  3 ,  0 : 0
i =  4 ,  0 : 0
i =  5 ,  17 : 17
i =  6 ,  64 : 64
i =  7 ,  234 : 234
i =  8 ,  529 : 529
i =  9 ,  1234 : 1234
i =  10 ,  2246 : 2246
i =  11 ,  4263 : 4263
i =  12 ,  6797 : 6797
CPU times: user 1min 53s, sys: 375 ms, total: 1min 54s
Wall time: 1min 53s


In [24]:
# The matrices S, S^(-1), and R generate the group SL2Z
S = Matrix(ZZ, [[1,1],[0,1]])
R = Matrix(ZZ,[[0,-1],[1,0]])

def SL2Z_Action(ori, A):
    '''
    Func: The SL2Z_Action function takes an origami ori and a 2x2 matrix A with integer coefficients as arguments.
    It tests whether A is an element of SL2Z, and if so it returns the image of ori under the action of A.
    If A is not an element of SL2Z, it returns an error message.
    
    Args:
    ori: The origami to be acted on.
    A: The 2x2 matrix with integer coefficients acting on ori.
    
    Return:
    An error string if A is not an SL2Z.
    The image of ori under the action of A otherwise.
    '''
    
    # Tests whether A is in SL2Z
    if det(A) != 1:
        return "Error: The input matrix is not an element of SL2Z"
        
    # List that will hold generators that produce A when multiplied in the order they appear
    # Temporary matrix that we will update as we reduce A to the identity using generators
    generators = []
    A_temp = A
    
    # Reduce A_temp to the identity by multiplying on the left by S, S^(-1), and R repeatedly
    # Simultaneously add the inverse of the matrix that is being multiplied to the end of list of generators
    while A_temp[1][0] != 0:
        if abs(A_temp[0][0])>=abs(A_temp[1][0]):
            if sgn(A_temp[0][0]) == sgn(A_temp[1][0]):
                A_temp = S^(-1)*A_temp
                generators = generators + [S]
            else:
                A_temp = S*A_temp
                generators = generators + [S^(-1)]
        else:
            A_temp = R*A_temp
            generators = generators + [R,R,R]
    if A_temp[0][0] == 1:
        while A_temp[0][1] > 0:
            A_temp = S^(-1)*A_temp
            generators = generators + [S]
        while A_temp[0][1] < 0:
            A_temp = S*A_temp
            generators = generators + [S^(-1)]
    else:
        while A_temp[0][1] < 0:
            A_temp = S^(-1)*A_temp
            generators = generators + [S]
        while A_temp[0][1] > 0:
            A_temp = S*A_temp
            generators = generators + [S^(-1)]
        A_temp = R*R*A_temp
        generators = generators + [R,R]
            
    # Origami that will be returned after the action of A is applied
    A_times_ori = ori
    
    # Apply generators in list in reverse order because A is multiplied on the left of ori
    for generator in reversed(generators):
        if generator == R:
            A_times_ori = A_times_ori.horizontal_symmetry().mirror()
        elif generator == S:
            A_times_ori = A_times_ori.horizontal_twist(1)
        else:
            A_times_ori = A_times_ori.horizontal_twist(-1)
            
    return A_times_ori

In [25]:
def count_SL_orbits(oris):
    '''
    Func: The count_SL_orbits function takes a list of origamis and returns the
    number of distinct SL2Z orbits represented by the origamis in the list.
    
    Args:
    oris: A list of origamis.
    
    Return:
    A list of lists or origamis, where each list of origamis represents a distinct SL2Z orbit.
    '''
    orbits = []
    for ori in oris:
        # Determine if ori is represented in any of the orbits already counted
        inOrbits = False
        for orbit in orbits:
            if inOrbits:
                break
            for comparison_ori in orbit:
                if ori.is_isomorphic(comparison_ori):
                    inOrbits = True
                    break
        
        # If not, then add orbit of ori to list of orbits
        if not inOrbits:
            orbit = []
            for A in ori.veech_group().coset_reps():
                A_inv = A^(-1) # Use inverse of A because coset_reps() gives right coset reps
                A_inv_times_ori = SL2Z_Action(ori, A_inv)
                orbit.append(A_inv_times_ori)
            orbits.append(orbit)
    return orbits

In [26]:
for i in range(15):
    print("i = ", i, ": ", len(count_SL_orbits(enum_H4(i))))

i =  0 :  0
i =  1 :  0
i =  2 :  0
i =  3 :  0
i =  4 :  0
i =  5 :  2
i =  6 :  4
i =  7 :  3
i =  8 :  3
i =  9 :  4
i =  10 :  7
i =  11 :  4
i =  12 :  9
i =  13 :  4
i =  14 :  9
